# Load MOdel

In [ ]:
!pip install vllm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.4/326.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 847.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/2

In [4]:
model ="lmsys/vicuna-13b-v1.3"

In [9]:
import os
os.environ["VLLM_LOGGING_LEVEL"] = "DEBUG"  # Enable verbose logging for debugging
# Print environment variables to check if VLLM_LOGGING_LEVEL is set correctly
for k, v in os.environ.items():
    if k.startswith("VLLM_"):
        print(f"{k}={v}")

VLLM_LOGGING_LEVEL=DEBUG


In [5]:
from vllm import LLM, SamplingParams

In [7]:
prompts = [

           "hello, how are you today",
]
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)

In [ ]:
llm = LLM(model=model, device="cpu", disable_async_output_proc=True)

In [ ]:
outputs = llm.generate(prompts, sampling_params)

# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

In [ ]:
import json
import textwrap

# def get_prompt(instruction):
#     prompt_template = "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n###Instruction\n{instruction}\n\n### Response\n"
#     return prompt_template.format(instruction=instruction)

system_prompt = "A chat between a curious user and an artificial intelligence assistant. \nThe assistant gives helpful, detailed, and polite answers to the user's questions."

addon_prompt = ""
# USER: What is 4x8?
# ASSISTANT:
def get_prompt(human_prompt):
    # prompt_template=f"{human_prompt}"
    prompt_template=f"{system_prompt}\n{addon_prompt} \n\nUSER: {human_prompt} \nASSISTANT: "
    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")

def generate(text):
    prompt = get_prompt(text)
    sampling_params = SamplingParams(
                                    max_tokens=512,
                                    # do_sample=True,
                                    temperature=0.7,
                                    top_p =0.95,
                                    top_k =  50,
                                    # eos_token_id=tokenizer.eos_token_id,
                                    # pad_token_id=tokenizer.pad_token_id,
                                     )
    outputs = llm.generate([prompt], sampling_params)
    return outputs

def parse_text(output):
        generated_text = output[0].outputs[0].text
        wrapped_text = textwrap.fill(generated_text, width=100)
        print(wrapped_text +'\n\n')
        # return assistant_text
        # prompt = output.prompt


In [ ]:
%%time
prompt = 'What are the differences between alpacas, vicunas and llamas?'
generated_text = generate(prompt)
final_text = parse_text(generated_text)
print(final_text)

In [ ]:
%%time
prompt = 'What is the capital of England?'
generated_text = generate(prompt)
parse_text(generated_text)

In [ ]:
%%time
prompt = 'Write an email to Sam Altman giving reasons to open source GPT-4'
generated_text = generate(prompt)
parse_text(generated_text)